In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd
import torch
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
import torchvision.transforms as transforms
import numpy as np

In [ ]:
video_name= "b607dd6a-1d8f33b9"
video_dir = "/home/fullsuper/Rex/dataset/bdd100k/bdd100k/videos/val/"
label_output_folder = "./"

In [2]:
model = YOLO('yolov8x.pt')

In [3]:
# video_path = "/home/fullsuper/Rex/dataset/single_car/single_car.mp4"
video_path = video_dir + video_name + ".mov"
test_frames = []

cap = cv2.VideoCapture(video_path) # origin frame rate is 30 fps
exist_flag = True
while exist_flag:
    ret, frame = cap.read()
    if ret:
        test_frames.append(cv2.resize(frame,(416,416),interpolation = cv2.INTER_AREA))
    
    if ret:
        exist_flag = True
    else:
        exist_flag = False
    
cap.release()

print(len(test_frames))

1203


In [4]:
results = model.predict(test_frames[0])
for result in results:
    print(result.boxes.cls )
    print(result.boxes.xyxy)


0: 640x640 8 cars, 2 buss, 2 trucks, 25.9ms
Speed: 15.4ms preprocess, 25.9ms inference, 91.8ms postprocess per image at shape (1, 3, 640, 640)
tensor([2., 2., 2., 2., 2., 7., 2., 5., 2., 7., 5., 2.], device='cuda:0')
tensor([[6.1135e+01, 2.2951e+02, 1.0859e+02, 2.8369e+02],
        [1.0145e+02, 2.3640e+02, 1.3116e+02, 2.7166e+02],
        [1.4383e+02, 2.3585e+02, 1.7379e+02, 2.7297e+02],
        [2.0927e+02, 2.5019e+02, 2.2478e+02, 2.6953e+02],
        [1.2277e-01, 1.9494e+02, 7.1390e+01, 3.0119e+02],
        [1.6888e+02, 2.3317e+02, 2.1050e+02, 2.9646e+02],
        [2.1964e+02, 2.5872e+02, 2.5564e+02, 2.9473e+02],
        [1.6892e+02, 2.3302e+02, 2.1047e+02, 2.9648e+02],
        [2.5538e+02, 2.4938e+02, 2.8998e+02, 2.9655e+02],
        [1.2714e-01, 1.9488e+02, 7.1421e+01, 3.0119e+02],
        [2.5541e+02, 2.4960e+02, 2.8975e+02, 2.9640e+02],
        [2.2545e+02, 2.5058e+02, 2.5291e+02, 2.6527e+02]], device='cuda:0')


In [5]:
len(result.boxes.cls)

12

In [6]:
def convert_rgb_frame_to_tensor(image):
    img_size = 416
    # Configure input
    input_img = transforms.Compose([
    DEFAULT_TRANSFORMS,
    Resize(img_size)])(
        (image, np.zeros((1, 5))))[0].unsqueeze(0)
    input_img = input_img.cuda()

    return input_img

In [7]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

In [8]:
df_result = pd.DataFrame(columns=["video_id","frame_id", 'class_id',"predict_acc","xmin","ymin","xmax","ymax"])
indexes = []
time_measurement = []

for index in range(len(test_frames)):
# for index in range(1):
    indexes.append(index)
    frame = test_frames[index]

    input_tensor = convert_rgb_frame_to_tensor(frame)
    
    start.record()
    results = model.predict(input_tensor)
    end.record()
    # results = results.cpu()
    torch.cuda.synchronize()
    time_measurement.append(start.elapsed_time(end))
    # [[x1, y1, x2, y2, confidence, class]]
    if len(results)!= 0 :
        for result in results:
            for i in range(len(result.boxes.cls)):
                df2 = pd.DataFrame([[video_name,
                                    index, 
                                    result.boxes.cls[i].item(),
                                    result.boxes.conf[i].item(),
                                    result.boxes.xyxy[i][0].item(),
                                    result.boxes.xyxy[i][1].item(),
                                    result.boxes.xyxy[i][2].item(),
                                    result.boxes.xyxy[i][3].item()]], columns=["video_id","frame_id", 'class_id',"predict_acc","xmin","ymin","xmax","ymax"])
                df_result=pd.concat([df_result,df2])
    else:
        df2 = pd.DataFrame([[video_name,index, -1,-1,-1,-1,-1,-1]], columns=["video_id","frame_id", 'class_id',"predict_acc","xmin","ymin","xmax","ymax"])
        df_result=pd.concat([df_result,df2])


0: 416x416 7 cars, 1 truck, 84.9ms
Speed: 0.0ms preprocess, 84.9ms inference, 2.3ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 7 cars, 1 truck, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 2.1ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 7 cars, 1 truck, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 7 cars, 1 truck, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 7 cars, 1 truck, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 8 cars, 1 truck, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 person, 8 cars, 1 truck, 14.9ms
Speed: 0.0ms preprocess, 14.9ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 person, 8 cars, 1 truck, 14.9ms
Speed: 0.0ms 

In [9]:
df_result.to_csv(label_output_folder+video_name+".csv",index=False)
# time_df = pd.DataFrame(list(zip(indexes, time_measurement)),columns =['index', 'elapsed'])
# time_df.to_csv("../generated_reference_yolov8/time.csv")